In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate
from dotenv import load_dotenv
import os
load_dotenv()
secret_value_0 = os.getenv("HF_TOKEN")
secret_value_1 = os.getenv("NGROK_KEY")
model_chat = "meta-llama/Llama-2-13b-chat-hf"
tokenizer_chat = AutoTokenizer.from_pretrained(model_chat, token=secret_value_0)
pipeline_chat = transformers.pipeline(
    "text-generation",
    model=model_chat,
    torch_dtype=torch.float16,
    device_map="auto",
    token=secret_value_0,
)

In [ ]:
from tavily import TavilyClient
secret_value_3 = os.getenv("TAVILY_API_KEY")
tavily_cli = TavilyClient(api_key=secret_value_3)

In [ ]:
import redis
secret_value_4 = os.getenv("UPSTASH_PD")
redis_client = redis.StrictRedis(host="usw1-organic-yak-33107.upstash.io", port='33107', password=secret_value_4, decode_responses=True)

In [ ]:
try:
    ngrok = redis_client.get('ngrok_link')
except Exception as e:
    link = ""

In [ ]:
def get_ques(in_s, out_s, query):
    prompt = """
    <s>[INST] <<SYS>>
    You are a helpful assistant that helps users generate questions. You will receive one question, and the question will be in JSON format. It will contain the question stem itself, no more than 4 options to answer the question, the correct option for the particular question, and the subject or topic related to the question.
    You will do your best to generate one question, which should be similar in topic, style and difficulty of the question provided to you. It should be in JSON Format, and provide the question stem itself, the options, and the correct option to select.
    You should ONLY have the JSON dictionary in your response.
    <</SYS>>

    {} [/INST] {} </s><s>[INST] {} [/INST]
    """.format(in_s, out_s, query)
    result = pipeline_chat(prompt)
    return result
def get_ans(in_s, out_s, query):
    prompt = """
    <s>[INST] <<SYS>>
    You are a helpful assistant that helps label correct answers for question. You will receive one question, and the question will be in JSON format. It will contain the question stem itself, no more than 4 options to answer the question, and information to help you answer the question.
    You will do your best to match the correct option. Your answer should be in JSON format, providing the correct answer. If the information provided states that the correct answer is something, not in the options, change one of the options to accomodate for the correct option.
    You should ONLY have the JSON dictionary in your response. The answer you provide should only be equal to A, B, C, D nothing else!
    <</SYS>>

    {} [/INST] {} </s><s>[INST] {} [/INST]
    """.format(in_s, out_s, query)
    result = pipeline_chat(prompt)
    return result

In [ ]:
sample_in="{'topic': 'Evolution', 'question': 'Man's evolution is defined as:', 'options': ['Dysversion', 'Anthropogenesis', 'Convergence', 'ontogenesis'], 'answer': 'B'}"
sample_out="{'question': 'Which of the following mechanisms is NOT a driving force of evolution?', 'options': ['Natural selection', 'Genetic drift', 'Lamarckism', 'Geographical barriers'], 'answer': 'C'}"
ques_in="{'topic': 'Kinematics', 'question': 'Which of the following values is a vector?', 'options': ['Speed', 'Distance', 'Position', 'Velocity'], 'info': 'D. Velocity is a vector, as it has a direction'}"
ques_out="{'topic': 'Kinematics', 'question': 'Which of the following values is a vector?', 'options': ['Speed', 'Distance', 'Position', 'Velocity'], 'info': 'D. Velocity is a vector, as it has a direction', 'answer': 'D'}"

In [ ]:
from flask import Flask, jsonify, request
import threading
from pyngrok import ngrok
import json
from flask_cors import CORS
import re
app = Flask(__name__)
CORS(app)
@app.route('/run', methods=['GET'])
def hello_world():
    try:
        data_in = request.args.get('msg')
        data_out = get_ques(sample_in, sample_out, data_in)[0]["generated_text"].split("[/INST]\n     ")[1]
        pattern = r"(?<=\W)'(.*?)'(?=\W)"
        output_string = re.sub(pattern, lambda x: f'"{x.group(1)}"', data_out)
        dict_ans = json.loads(output_string)
        query_tavily = dict_ans["question"] + ": "
        opt_str = "ABCD"
        for n in range(4):
            query_tavily += opt_str[n] + ". " + dict_ans["options"][n] + "  "
        context = tavily_cli.qna_search(query=query_tavily[:-2])
        try:
            del dict_ans["answer"]
        except Exception as e:
            pass
        dict_ans["info"] = context
        final_response = get_ans(ques_in, ques_out, dict_ans)[0]["generated_text"].split("[/INST]\n     ")[1]
        return jsonify({"output": final_response})
    except Exception as e:
        print(e)

def run_flask():
    app.run(port=6060)

ngrok_auth_token = secret_value_1
ngrok.set_auth_token(ngrok_auth_token)
public_url = ngrok.connect(6060)
threading.Thread(target=run_flask).start()
pattern_link = r"https?://\S+"
matches = str(re.findall(pattern_link, str(public_url))[0])[:-1]
redis_client.set('ngrok_link', matches+"/")
print(f"Access your Flask app at: {public_url}")